# metaprogramming

For example, if we want to write a "super duper debugger", that prints out every function call and the arguments it was called with, we can easily modify (decorate) any function we want to "debug" without modifying the function body directly:

In [1]:
from functools import wraps

def debugger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        print(f'{fn.__qualname__}', args, kwargs)
        return fn(*args, **kwargs)
    return inner

In [2]:
@debugger
def func_1(*args, **kwargs):
    pass

@debugger
def func_2(*args, **kwargs):
    pass

In [3]:
func_1(10, 20, kw='a')

func_1 (10, 20) {'kw': 'a'}


In [4]:
func_2(10)

func_2 (10,) {}


In [5]:
class IntegerField:
    def __set_name__(self, owner, name):
        self.name = name
        
    def __get__(self, instance, owner):
        return instance.__dict__.get(self.name, None)
    
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError('Must be an integer.')
        instance.__dict__[self.name] = value

In [6]:
class Point:
    x = IntegerField()
    y = IntegerField()
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [7]:
p = Point(10, 20)

In [8]:
p.x, p.y

(10, 20)

`__new__` method

In [9]:
class Point:
    pass

In [10]:
p=object.__new__(Point)

In [11]:
type(p)

__main__.Point

In [12]:
class Point:
    def __init__(self, x,y):
        self.x =x
        self.y =y

In [13]:
p=object.__new__(Point)
p.__init__(10, 20)

In [14]:
p.__dict__

{'x': 10, 'y': 20}

In [15]:
class Point:
    def __new__(cls, x, y):
        print('Creating instance...', x, y)
        instance = object.__new__(cls)  # delegate to object.__new__
        return instance  # don't forget to return the new instance!
    
    def __init__(self, x, y):
        print('Initializing instance...', x, y)
        self.x = x
        self.y = y

In [16]:
p = Point(10, 20)

Creating instance... 10 20
Initializing instance... 10 20


In [17]:
class Squared(int):
    def __new__(cls, x):
        return super().__new__(cls, x**2)

In [18]:
s=Squared(3)
type(s), s

(__main__.Squared, 9)

Trying to do this using `__init__` would not work - the built-in `__init__` for integers does not actually do anything, and does not allow for an argument to be passed:

In [19]:
class Squared(int):
    def __init__(self, x):
        print('calling init...', x)
        super().__init__(x**2)

In [20]:
try:
    result = Squared(3)
except TypeError as ex:
    print(ex)

calling init... 3
object.__init__() takes exactly one argument (the instance to initialize)


Most often when we override the `__new__` method we use delegation to the parent class to do some of the work. But of course, as we saw just now we don't have to, we can just use `object.__new__` directly. 

In [21]:
class Person:
    def __new__(cls, name):
        print(f'Person: Instantiating {cls.__name__}...')
        instance = object.__new__(cls)
        return instance
        
    def __init__(self, name):
        print(f'Person: Initializing instance...')
        self.name = name

In [22]:
p = Person('Guido')

Person: Instantiating Person...
Person: Initializing instance...


but the problem does not play well with inheritence

In [23]:
class Student(Person):
    def __new__(cls, name, major):
        print(f'student initializing {cls.__name__}', name , major)
        instance=object.__new__(cls)
        return instance
    
    def __init__(self, name, major):
        print(f'Student initializing instance ', name, major)
        super().__init__(name)
        self.major=major

In [24]:
s=Student('narain', 'btech')

student initializing Student narain btech
Student initializing instance  narain btech
Person: Initializing instance...


In [25]:
class Person:
    def __new__(cls, name):
        print(f'Person: Instantiating {cls.__name__}...')
        instance = super().__new__(cls)
        return instance
        
    def __init__(self, name):
        print(f'Person: Initializing instance...')
        self.name = name
        
class Student(Person):
    def __new__(cls, name, major):
        print(f'Student: Instantiating {cls.__name__}...')
        instance = super().__new__(cls, name)
        return instance
    
    def __init__(self, name, major):
        print(f'Student: Initializing instance...')
        super().__init__(name)
        self.major = major

In [26]:
s=Student('narain', 'btech')

Student: Instantiating Student...
Person: Instantiating Student...
Student: Initializing instance...
Person: Initializing instance...


In [27]:
class Square:
    def __new__(cls, w, l):
        cls.area= lambda self:self.w*self.l
        instance=super().__new__(cls)
        return instance
    
    def __init__(self, w, l):
        self.w=w
        self.l=l

In [28]:
s=Square(11, 23)

In [29]:
s.area()

253

In [30]:
class Square:
    def __new__(cls, w, l):
        setattr(cls, 'area', lambda self: self.w*self.l)
        instance=super().__new__(cls)
        instance.w=w
        instance.l=l
        return instance

In [31]:
s= Square(11, 23)
s.area()

253

In [32]:
s.__dict__

{'w': 11, 'l': 23}

In [33]:
s=Square.__new__(Square, 3,3)
s.area()

9

In [34]:
s.__dict__

{'w': 3, 'l': 3}

In [35]:
class Person:
    def __new__(cls, name):
        print(f'creating instance of {cls.__name__}')
        instance=str(name)
        return instance
    
    def __init__(self, name):
        print(f'initializing the variables of class {self.__name__} ={name}')
        self.name=name

In [36]:
p=Person('narain')

creating instance of Person


In [37]:
type(p), p

(str, 'narain')

In [38]:
class Person:
    def __new__(cls, name, age):
        print(f'creating instance of {cls.__name__}')
        instance=super().__new__(cls)
        instance.name=name
        instance.age=age
        instance.valid= True if instance.age>18 else False
        return instance

In [39]:
p=Person('narain', 21)
p.__dict__

creating instance of Person


{'name': 'narain', 'age': 21, 'valid': True}

In [40]:
p=Person('isai', 8)
p.__dict__

creating instance of Person


{'name': 'isai', 'age': 8, 'valid': False}

## creation of classes

In [41]:
from math import pi

class Circle:
    def __init__(self, x, y,r):
        self.x=x
        self.y=y
        self.r=r
        
    def area(self)-> float:
        return pi*r*r

In [42]:
type(Circle)

type

There are four main steps involved with creating instances of a class:

1. The class body is extracted - think of it as just a lump of text that contains code.
2. The class dictionary (used for the **class** state) is created for the class namespace
3. The body (extracted in 1), is executed in the class namespace (created in 2), thereby populating the class dictionary (in this case with two symbols, `__init__` and `area`)
4. A new `type` **instance** is constructed using the name of the class, the base classes (remember Python supports multiple inheritance), and that dictionary.

In [43]:
namespc={}

exec('''
area=10
perimeter=20
''', globals(), namespc)

In [44]:
namespc, namespc['area'], namespc['perimeter']

({'area': 10, 'perimeter': 20}, 10, 20)

In [45]:
exec('''
def add(a, b):
    return a + b
    
def mul(a, b):
    return a * b
''', globals(), namespc)

In [46]:
del Circle

In [47]:
class_name='Circle'

class_body='''
def __init__(self, x, y, r):
    self.x=x
    self.y=y
    self.r=r
    
def area(self):
    return pi*self.r*self.r'''

In [48]:
class_bases=()
class_dict={}

In [49]:
exec(class_body, globals(), class_dict)

In [50]:
class_dict

{'__init__': <function __main__.__init__(self, x, y, r)>,
 'area': <function __main__.area(self)>}

In [51]:
Circle=type(class_name, class_bases, class_dict)

In [52]:
Circle

__main__.Circle

In [53]:
type(Circle)

type

In [54]:
Circle.__dict__

mappingproxy({'__init__': <function __main__.__init__(self, x, y, r)>,
              'area': <function __main__.area(self)>,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Circle' objects>,
              '__weakref__': <attribute '__weakref__' of 'Circle' objects>,
              '__doc__': None})

In [55]:
Circle(0,0,3).area()

28.274333882308138

### to create a class from the callable type 

In [56]:
Circle = type(class_name, class_bases, class_dict)

In [57]:
type(Circle)

type

In [58]:
class CustomType(type):
    def __new__(cls, name, bases, class_dict):
        print('customized type creation')
        cls_obj= super().__new__(cls, name, bases, class_dict)
        cls_obj.circle= lambda self: 2* pi* self.r
        return cls_obj

In [60]:
Circle = CustomType('Circle', (), class_dict)

customized type creation


In [61]:
type(Circle)

__main__.CustomType

In [68]:
class CustomType(type):
    def __new__(mcls, name, bases, class_dict):
        print(f'Using custom metaclass {mcls} to create class {name}...')
        cls_obj = super().__new__(mcls, name, bases, class_dict)
        cls_obj.circ = lambda self: 2 * pi * self.r
        return cls_obj

In [71]:
class Circle(metaclass=CustomType):
    def __init__(self, x, y, r):
        self.x = x
        self.y = y
        self.r = r
        
    def area(self):
        return pi * self.r ** 2

Using custom metaclass <class '__main__.CustomType'> to create class Circle...


In [72]:
c = Circle(0, 0, 1)
print(c.area())
print(c.circ())

3.141592653589793
6.283185307179586


## class decorators

In [73]:
def savings(cls):
    cls.account_type= 'savings'
    return cls

def checking(cls):
    cls.account_type= 'chacking'
    return cls

In [75]:
class Account:
    pass

@savings
class BankAcc1(Account):
    pass

@savings
class BankAcc2(Account):
    pass

@checking
class BankSav1(Account):
    pass

@checking
class BankSav2(Account):
    pass

In [77]:
BankAcc1.account_type

'savings'

In [78]:
def account_type(type_):
    def dec(cls):
        cls.account_type=type_
        return cls
    return dec

In [79]:
@account_type('savings')
class BankAcc3(Account):
    pass

BankAcc3.account_type

'savings'

In [82]:
from functools import wraps

def func_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner    

In [83]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
    return cls

In [84]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def greet(self):
        return f'Hello, my name is {self.name} and I am {self.age}'

decorating: <class '__main__.Person'> __init__
decorating: <class '__main__.Person'> greet
